In [ ]:
import itertools

In [ ]:
import sys
sys.path.append('./..')

In [ ]:
import numpy as np
from embedml.nn import CrossEntropy
from embedml.util import Dataset
from embedml.util import DataLoader
from collections import namedtuple

In [ ]:
from models.gpt import GPT
from collections import namedtuple

In [ ]:
class data(Dataset):
    def __init__(self, ndigit=3, train=True):
        self.train = train
        self.ndigit = ndigit
        self._generate()
        pass
    def _generate(self):
        self.p = itertools.product(np.random.permutation(10 ** self.ndigit), np.random.permutation(10 ** self.ndigit))

    @property
    def get_vocab_size(self):
        return 10 # digits 0..9
    @property
    def get_block_size(self):
        return 3 * self.ndigit + 1 - 1
    def __len__(self):
        return 10 ** (self.ndigit * 2) * 8 // 10 if self.train else 10 ** (self.ndigit * 2) * 2 // 10
    
    def __getitem__(self, idx):
        if isinstance(idx, int) and idx < self.__len__():
            rng = [idx]
            
        if idx[-1] < self.__len__(): 
            try:
                ab = [next(self.p) for j in idx]
            except StopIteration:
                raise
            
            fmt0, fmt1 = f'0{self.ndigit}', f'0{self.ndigit + 1}'
            ls =[]
            for a, b in ab:
                ret = f"{a:{fmt0}}{b:{fmt0}}{f'{a+b:{fmt1}}'[::-1]}"
                ls.append([int(s) for s in ret])
        
            x = np.array(list(map(lambda x: x[:-1], ls)), dtype=int)
            y = np.array(list(map(lambda x: x[1:], ls)), dtype=int) 
            y[:,:self.ndigit*2-1] = -1 
                
        
        else:
            self._generate()
            raise StopIteration
        return x, y
        

In [ ]:
d = data(ndigit=2, train=True)

In [ ]:
dl = DataLoader(d, batch_size=8)

In [ ]:
config = namedtuple(
        'config',
        ['n_layer', 'n_embd', 'n_head', 'resid_pdrop', 'attn_pdrop', 'vocab_size', 'block_size', 'embd_pdrop'],
        defaults=[1, 48, 3, 0.1, 0.1, 10, 6, 0.1]
    )

In [ ]:
gpt = GPT(config)

In [ ]:
x, y = next(dl)

In [ ]:
t = gpt(x)

In [ ]:
criterion= CrossEntropy(num_class=10) # TODO: ignore -1

In [ ]:
l = criterion(t.reshape((-1, 10)), y.reshape((1, -1)))

In [ ]:
l.backward()